# Sentiment Analysis with TF-IDF text representation

## Our Dataset

This dataset describes the contents of the heart-disease diagnosis.

The dataset in this study is from [Kaggle](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data), which is called Twitter US Airline Sentiment.

- Dataset: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data

## Variable Table

| Original Dataset             | Data Type     | Description    |                                                         
|------------------------------|---------------|---------------------------------------------------------------------------------------------|
| tweet_id                     | ID            | A unique identifier for each tweet.                                                         | 
| airline_sentiment            | Categorical   | The sentiment expressed in the tweet (positive, neutral, negative).                         | 
| airline_sentiment_confidence | Numerical     | Confidence score in the sentiment label (0 to 1).                                           | 
| negativereason               | Categorical   | Reason for negative sentiment (e.g., "Late Flight", "Customer Service Issue").              | 
| negativereason_confidence    | Numerical     | Confidence score in the negative reason label (0 to 1).                                     | 
| airline                      | Categorical   | The airline mentioned in the tweet (e.g., United, Delta, etc.).                             | 
| airline_sentiment_gold       | Categorical   | Sentiment label by trusted annotator (gold standard).                                       | 
| name                         | Text          | Name of the user who posted the tweet.                                                      | 
| negativereason_gold          | Categorical   | Negative reason label by trusted annotator (gold standard).                                 | 
| retweet_count                | Numerical     | Number of times the tweet was retweeted.                                                    | 
| text                         | Text          | The full content of the tweet.                                                              | 
| tweet_coord                  | Geospatial    | Latitude and longitude coordinates where the tweet was posted, if available.                | 
| tweet_created                | Datetime      | Timestamp when the tweet was created.                                                       | 
| tweet_location               | Text          | Location specified in the user's profile.                                                   | 
| user_timezone                | Categorical   | Time zone specified in the user's profile.                                                  | 

<br/>

## Data Used for Modeling

| Feature                      | Data Type   | Description  |
|-----------------------------|-------------|--------------|
| **Target Variable: `encoded_sentiment`** | Categorical | This is an engineered variable derived from `airline_sentiment` for multi-class sentiment classification. It encodes sentiment as: 0 = Negative, 1 = Neutral, 2 = Positive. |
| **Feature: `text`**         | Text        | Contains consumer tweets about U.S. airlines. This field undergoes preprocessing, including removal of URLs and mentions (`@`), stopword removal, and stemming. |


<br/>

# 1. Load Data

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../../data/tweets.csv.gz", compression="gzip")

In [2]:
# Show basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [3]:
# Show the first few rows
df.head(3)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)


# 2. Data Preprocessing

## 2.1 Handle Duplicates

In [4]:
# Check for duplicate rows
duplicate_rows = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Drop duplicate rows
df.drop_duplicates(inplace=True)

Number of duplicate rows: 36


In [5]:
# Confirm the shape after removal
print(f"Shape after dropping duplicates: {df.shape}")

Shape after dropping duplicates: (14604, 15)


## 2.2 Handle Missing Values

In [6]:
# Check for missing values for each variables in the dataset
print("\nMissing values count for each variables:")
print("-------------------------------------------")
print(df.isnull().sum())

print("""\n\n**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values""")


Missing values count for each variables:
-------------------------------------------
tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5445
negativereason_confidence        4101
airline                             0
airline_sentiment_gold          14564
name                                0
negativereason_gold             14572
retweet_count                       0
text                                0
tweet_coord                     13589
tweet_created                       0
tweet_location                   4723
user_timezone                    4814
dtype: int64


**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values


## 2.3 Text Processing

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk


nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\tys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tys\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 2.3.1 Feature Engineering

For feature engineering in sentiment analysis, we will perform the following steps:

- `Tokenization`: Breaking the text into individual words or tokens.
- `Stopwords Removal`: Eliminating common words (e.g., "the", "is", "and") that don't contribute meaningful information.
- `Stemming`: Reducing words to their base or root form (e.g., "running" → "run", "happily" → "happy")."

In [8]:
import re

# Initialize stopwords, stemmer
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Step 1: Lowercase and clean the text
def clean_text(text):
    text = text.lower()                                 # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text) # Remove URLs
    text = re.sub(r'@\w+', '', text)                    # Remove mentions
    # text = re.sub(r'#\w+', '', text)                    # NOTE: Do not remove hashtags, 
                                                                # as there is a lot of hashtags with sentiment indication, 
                                                                # such as '#thankyou', '#happycustomer', etc...
    text = re.sub(r'[^a-z\s]', '', text)                # Remove numbers and punctuation
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Step 2: Tokenization
def tokenize_text(text):
    return word_tokenize(text)

# Step 3: Remove stopwords
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

# Step 4: Apply Stemming
def apply_stemming(tokens):
    return " ".join([stemmer.stem(word) for word in tokens])

# Copy only the 'text' column to df_copy
df2 = df[['text', 'airline_sentiment']].copy()

# rule-based text processing for removing URL, twitter username and punctutations
df2['clean_text'] = df2['text'].apply(clean_text)

# procesed text after stemming
df2['tokens'] = df2['clean_text'].apply(tokenize_text)
df2['removed_stopwords'] = df2['tokens'].apply(remove_stopwords)
df2['stemmed'] = df2['removed_stopwords'].apply(apply_stemming)


In [9]:
df2.shape

(14604, 6)

In [10]:
# Check for missing values per column
print("\nMissing values per column:")
print(df2.isnull().sum())


Missing values per column:
text                 0
airline_sentiment    0
clean_text           0
tokens               0
removed_stopwords    0
stemmed              0
dtype: int64


### 2.3.2 Target Engineering

We will convert the 'airline_sentiment' column into numerical values to use it as the target variable in our model, where `negative` = 0, `neutral` = 1, and `positive` = 2 

In [11]:
print("The target variable contains unique values of: ", df2['airline_sentiment'].unique(), 
      "which we are going to map it into 0, 1 and 2 respectively")

# Encode the sentiment column
df2['encoded_sentiment'] = df2['airline_sentiment'].map({"negative": 0, "neutral": 1, "positive": 2})


The target variable contains unique values of:  ['neutral' 'positive' 'negative'] which we are going to map it into 0, 1 and 2 respectively


## 3.0 Data Preparation for Modeling

In [12]:
# Split into features (X) and target labels (y)
X = df2['stemmed']
y = df2['encoded_sentiment']

### 3.1 Train test valid split with stratified sampling

In [13]:
from sklearn.model_selection import train_test_split

# Train test validation split with ration 70:15:15
X_train, X_temp, y_train, y_temp = train_test_split(X, y, 
                                        stratify=y,            
                                        train_size=0.7, 
                                        random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, 
                                        stratify=y_temp,            
                                        train_size=0.5, 
                                        random_state=42)

In [14]:
# Verify that class distribution is preserved after the train-test split (i.e., stratified correctly)

print("Train class distribution:")
print(y_train.value_counts(normalize=True))

print("\nValidation class distribution:")
print(y_valid.value_counts(normalize=True))

print("\nTest class distribution:")
print(y_test.value_counts(normalize=True))

print("""\n**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.""")

Train class distribution:
encoded_sentiment
0    0.627177
1    0.211602
2    0.161221
Name: proportion, dtype: float64

Validation class distribution:
encoded_sentiment
0    0.627111
1    0.211775
2    0.161114
Name: proportion, dtype: float64

Test class distribution:
encoded_sentiment
0    0.627111
1    0.211775
2    0.161114
Name: proportion, dtype: float64

**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.


### 3.2 Text Representation with Word2Vec

Word2Vec represents words as dense vectors in a continuous vector space, capturing semantic and syntactic relationships between words based on their context. Instead of counting how often a word appears, Word2Vec learns how words are used in similar contexts across a corpus. Words that appear in similar contexts will have similar vector representations, allowing the model to understand relationships like analogies or word similarities.

In this notebook, we will be using word2vec embedding with dimension size of 300.

In [15]:
import os
import pandas as pd
from gensim.models import Word2Vec 
from nltk.tokenize import word_tokenize
import numpy as np

# File paths
train_path = "../../data/word2vec_embedding_train.csv.gz"
valid_path = "../../data/word2vec_embedding_valid.csv.gz"
test_path = "../../data/word2vec_embedding_test.csv.gz"

# Check if files exist
if os.path.exists(train_path) and os.path.exists(valid_path):
    # Load precomputed embeddings
    X_train_vectorized = pd.read_csv(train_path, compression="gzip", index_col=False).values
    X_valid_vectorized = pd.read_csv(valid_path, compression="gzip", index_col=False).values
    
else:
    # Tokenize cleaned strings for Word2Vec
    tokenized_train = [word_tokenize(text) for text in X_train]
    tokenized_valid = [word_tokenize(text) for text in X_valid]
    tokenized_test = [word_tokenize(text) for text in X_test]

    # Train Word2Vec on all tokens
    w2v_model = Word2Vec(
        sentences=tokenized_train + tokenized_valid + tokenized_test, 
        vector_size=300, 
        window=5, 
        min_count=1, 
        workers=4
    )

    # Average vector generator
    def get_avg_w2v(tokens, model, k=300):
        vecs = [model.wv[word] for word in tokens if word in model.wv]
        return np.mean(vecs, axis=0) if vecs else np.zeros(k)

    # Create average embeddings
    X_train_vectorized = np.vstack([get_avg_w2v(t, w2v_model) for t in tokenized_train])
    X_valid_vectorized = np.vstack([get_avg_w2v(t, w2v_model) for t in tokenized_valid])
    X_test_vectorized = np.vstack([get_avg_w2v(t, w2v_model) for t in tokenized_test])

    # Save to CSV
    pd.DataFrame(X_train_vectorized).to_csv(train_path, index=False)
    pd.DataFrame(X_valid_vectorized).to_csv(valid_path, index=False)
    pd.DataFrame(X_test_vectorized).to_csv(test_path, index=False)


### 3.3 Handling class imbalance issue with SMOTE

In [16]:
from imblearn.over_sampling import SMOTE

# Handling imbalanced using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

In [17]:
# Check class distribution before and after applying SMOTE to confirm successful balancing

print("Class distribution in training set (before SMOTE):")
print(y_train.value_counts())

print("\nClass distribution in training set (after SMOTE):")
print(y_train_resampled.value_counts())

print("""\n**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.""")


Class distribution in training set (before SMOTE):
encoded_sentiment
0    6411
1    2163
2    1648
Name: count, dtype: int64

Class distribution in training set (after SMOTE):
encoded_sentiment
1    6411
0    6411
2    6411
Name: count, dtype: int64

**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.


## 4.0 Modeling

(i) Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the Logistic Regression model
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
lr_model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and valid sets
y_train_pred_lr = lr_model.predict(X_train_resampled)
y_valid_pred_lr = lr_model.predict(X_valid_vectorized)
y_test_pred_lr = lr_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_lr)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_lr)
test_accuracy = accuracy_score(y_test, y_test_pred_lr)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_lr, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_lr, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_lr, average='weighted')
test_precision = precision_score(y_test, y_test_pred_lr, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_lr, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_lr, average='weighted')
test_recall = recall_score(y_test, y_test_pred_lr, average='weighted')


# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Validation Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Validation F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Validation Precision:  {valid_precision:.4f}")
print(f"Test F1-score:  {test_precision:.4f}\n")


print(f"Train Recall: {train_recall:.4f}")
print(f"Validation Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_lr))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_lr))

Train Accuracy: 0.6211
Validation Accuracy:  0.6047
Test Accuracy:  0.5888

Train F1-score: 0.6221
Validation F1-score:  0.6243
Test F1-score:  0.6094

Train Precision: 0.6335
Validation Precision:  0.7003
Test F1-score:  0.6814

Train Recall: 0.6211
Validation Recall:  0.6047
Test Recall:  0.5888

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.85      0.58      0.69      1374
           1       0.37      0.67      0.48       464
           2       0.45      0.53      0.49       353

    accuracy                           0.59      2191
   macro avg       0.56      0.59      0.55      2191
weighted avg       0.68      0.59      0.61      2191

Confusion Matrix (Test):
[[791 421 162]
 [ 83 311  70]
 [ 61 104 188]]


(ii) Decision Tree

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the Decision Tree
dt_model = DecisionTreeClassifier(max_depth=100, 
                                  min_samples_split=10, 
                                  criterion='entropy', 
                                  min_samples_leaf=100,
                                  random_state=42)

# Train the model
dt_model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and test sets
y_train_pred_dt = dt_model.predict(X_train_resampled)
y_valid_pred_dt = dt_model.predict(X_valid_vectorized)
y_test_pred_dt = dt_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_dt)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_dt)
test_accuracy = accuracy_score(y_test, y_test_pred_dt)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_dt, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_dt, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_dt, average='weighted')
test_precision = precision_score(y_test, y_test_pred_dt, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_dt, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_dt, average='weighted')
test_recall = recall_score(y_test, y_test_pred_dt, average='weighted')

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Valid Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Valid F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Valid Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_dt))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_dt))

Train Accuracy: 0.6512
Valid Accuracy:  0.5678
Test Accuracy:  0.5582

Train F1-score: 0.6510
Valid F1-score:  0.5907
Test F1-score:  0.5806

Train Precision: 0.6527
Valid Precision:  0.6632
Valid Precision:  0.6632

Train Recall: 0.6512
Valid Recall:  0.5678
Test Recall:  0.5582

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.82      0.57      0.67      1374
           1       0.36      0.56      0.44       464
           2       0.36      0.53      0.43       353

    accuracy                           0.56      2191
   macro avg       0.51      0.55      0.51      2191
weighted avg       0.65      0.56      0.58      2191

Confusion Matrix (Test):
[[777 354 243]
 [114 258  92]
 [ 62 103 188]]


(iii) XGBoost

In [20]:
import os
import joblib
import xgboost as xgb
from sklearn.metrics import (f1_score, accuracy_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)

# Initialize the XGBoost Classifier
model_path = "../../models/word2vec_embedding_xgb_model.pkl"
if os.path.exists(model_path):
    print("Loading existing model...")
    xgb_model = joblib.load(model_path)
else:
    # Train the model
    xgb_model = xgb.XGBClassifier(max_depth=10,
                                random_state=42,
                                # Introduce randomness to make training faster and reduce overfitting
                                subsample=0.8, ## Uses 80% of the data for each tree.
                                colsample_bytree=0.8, ## Uses 80% of the features for each tree.
                                # the parameters below make the model trained faster by enabling parallelism
                                n_jobs = -1)
    xgb_model.fit(X_train_resampled, y_train_resampled)
    # Export and load previously trained model to avoid retraining every time
    joblib.dump(xgb_model, model_path)
    print("Model saved.")

# Predictions on training and test sets
y_train_pred_xgb = xgb_model.predict(X_train_resampled)
y_valid_pred_xgb = xgb_model.predict(X_valid_vectorized)
y_test_pred_xgb = xgb_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_xgb)
valid_accuracy = accuracy_score(y_valid, y_valid_pred_xgb)
test_accuracy = accuracy_score(y_test, y_test_pred_xgb)

# F1 scores
train_f1_score = f1_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_f1_score = f1_score(y_valid, y_valid_pred_xgb, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred_xgb, average='weighted')

# Precision scores
train_precision = precision_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_precision = precision_score(y_valid, y_valid_pred_xgb, average='weighted')
test_precision = precision_score(y_test, y_test_pred_xgb, average='weighted')

# Recall scores
train_recall = recall_score(y_train_resampled, y_train_pred_xgb, average='weighted')
valid_recall = recall_score(y_valid, y_valid_pred_xgb, average='weighted')
test_recall = recall_score(y_test, y_test_pred_xgb, average='weighted')

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Valid Accuracy:  {valid_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print(f"Train F1-score: {train_f1_score:.4f}")
print(f"Valid F1-score:  {valid_f1_score:.4f}")
print(f"Test F1-score:  {test_f1_score:.4f}\n")

print(f"Train Precision: {train_precision:.4f}")
print(f"Valid Precision:  {valid_precision:.4f}")
print(f"Test Precision:  {test_precision:.4f}\n")

print(f"Train Recall: {train_recall:.4f}")
print(f"Valid Recall:  {valid_recall:.4f}")
print(f"Test Recall:  {test_recall:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_xgb))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_xgb))


Model saved.
Train Accuracy: 0.9946
Valid Accuracy:  0.6764
Test Accuracy:  0.6737

Train F1-score: 0.9946
Valid F1-score:  0.6781
Test F1-score:  0.6729

Train Precision: 0.9947
Valid Precision:  0.6801
Test Precision:  0.6723

Train Recall: 0.9946
Valid Recall:  0.6764
Test Recall:  0.6737

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.78      0.79      0.78      1374
           1       0.46      0.46      0.46       464
           2       0.53      0.51      0.52       353

    accuracy                           0.67      2191
   macro avg       0.59      0.59      0.59      2191
weighted avg       0.67      0.67      0.67      2191

Confusion Matrix (Test):
[[1082  186  106]
 [ 195  214   55]
 [ 109   64  180]]
